In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
import re
os.chdir("../")

In [3]:
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 100)

## Analyze data

In [4]:
import pandas as pd
df = pd.read_parquet("data/prepared_data/prepared_df.parquet")

In [5]:
df

,place_id,review,question,tags,proba
0,1,Красивая набережная с барельефами отражающими историю города Тюмень Красивые виды Зимой строят с...,на что можно посмотреть в этом месте,[соревнование],"[0.625, 0.73]"
1,1,Красивая набережная с барельефами отражающими историю города Тюмень Красивые виды Зимой строят с...,как было в этом месте,[красивая набережная],"[0.722, 0.309]"
2,1,Красивая набережная с барельефами отражающими историю города Тюмень Красивые виды Зимой строят с...,что делать в этом месте,"[посмотреть, просто погулять]","[0.643, 0.516]"
3,1,Красивая набережная с барельефами отражающими историю города Тюмень Красивые виды Зимой строят с...,какие ожидания от этого места,"[посмотреть, просто погулять]","[0.671, 0.56]"
4,1,Красивая набережная с барельефами отражающими историю города Тюмень Красивые виды Зимой строят с...,для кого это место,"[посмотреть, просто погулять]","[0.487, 0.379]"
...,...,...,...,...,...
21980,199,Не адекват встречает на входе. Стоит малолетка и запускает бородатых и губастых малолеток. Хотел...,на что можно посмотреть в этом месте,[не адекват встречает на входе],"[0.412, 0.282]"
21981,199,Не адекват встречает на входе. Стоит малолетка и запускает бородатых и губастых малолеток. Хотел...,как было в этом месте,[],[]
21982,199,Не адекват встречает на входе. Стоит малолетка и запускает бородатых и губастых малолеток. Хотел...,что делать в этом месте,"[стоит малолетка, запускает бородатых, губастых малолеток]","[0.608, 0.392]"
21983,199,Не адекват встречает на входе. Стоит малолетка и запускает бородатых и губастых малолеток. Хотел...,какие ожидания от этого места,[хотели потратить],"[0.598, 0.3]"


---

In [6]:
from src.models import roberta
from nltk.corpus import stopwords

/home/dmitry/projects/tags-by-review-nlp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df = pd.read_parquet("data/intermediate_data/comments_tags.parquet").iloc[:10]

In [17]:
model = roberta.Pipeline()

In [18]:
config = roberta.config

In [19]:
questions = config["questions"]
proba = config["cut_off_proba"]

In [26]:
example = df.review_body[10]
print(example, "\n")
for question in questions:
    answer, probas = model.answer(example, question, 0.)

    if (
        not answer or
        answer == '' or
        answer == question.lower().replace('?', '')
    ):
        continue

    try:
        answer = answer.split(">")[1]
    except IndexError:
        pass
    # print(answer)

    stops = stopwords.words("russian")
    stops = [word for word in stops if word not in model._norm_words+["не", "на", "от", "а", "в", "за", "до", "с"]]

    corpus = answer.split(" ")
    text = " ".join([word for word in corpus if word not in stops])
    text_list = re.split(r', |\. |\.|,| а | и | просто | или |!|!!!| ! ', text)

    answer_cleared = text_list
    # print(text_list)

    parts = []
    roots_dep = []
    for answer in answer_cleared:
        document = model._nlp_model(answer)
        doc_dict = {}
        for token in document:
            if token.pos_ not in "PUNCT":
                doc_dict[token.text] = {
                    "lemma": token.lemma_,
                    "pos": token.pos_,
                    "dep": token.dep_,
                }
                if token.dep_ == "ROOT":
                    roots_dep.append(token.pos_)
        parts.append(doc_dict)
    
    print(parts)

    tags = []
    for part, root_dep in zip(parts, roots_dep):
        if part:
            tags.append(model._return_tag(part, root_dep))
    
    print(question, tags, "\n", sep = "\n")

KeyError: 10

: 